In [23]:
import torch
import torchaudio
from datasets import load_dataset
from IPython.display import Audio

from sam_audio import SAMAudio, SAMAudioProcessor

In [11]:
# Get an example audio from AudioCaps
dset = load_dataset(
    "parquet",
    data_files="hf://datasets/OpenSound/AudioCaps/data/test-00000-of-00041.parquet",
)
samples = dset["train"][8]["audio"].get_all_samples()
Audio(samples.data, rate=samples.sample_rate)

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SAMAudio.from_pretrained("facebook/sam-audio-3b").to(device).eval()
processor = SAMAudioProcessor.from_pretrained("facebook/sam-audio-3b")

In [25]:
wav = torchaudio.functional.resample(
    samples.data, samples.sample_rate, processor.audio_sampling_rate
)
wav = wav.mean(0, keepdim=True)
inputs = processor(
    audios=[wav], descriptions=["A horn honking"], anchors=[[["+", 6.3, 7.0]]]
).to(device)
with torch.inference_mode():
    result = model.separate(inputs)

In [28]:
Audio(result.target[0].cpu(), rate=processor.audio_sampling_rate)